# Forecast Analys ETH


# Först läs in data och visualisera

In [1]:
# !pip install datetime --user --upgrade pip
# !pip install pandas -U
# !pip install pandas_datareader -U
# !pip install matplotlib -U
# !pip install --user catboost -U
# !pip install yfinance --upgrade --no-cache-dir
# pip install yfinance --upgrade --no-cache-dir
# import numpy as np
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
# import pandas_datareader.data as web
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from catboost import CatBoostRegressor,Pool,utils

import time
plt.style.use('fivethirtyeight')
pd.set_option('display.width',200)

In [2]:
### Read in Ethereum ###
# data = web.DataReader(['ETH-USD','BTC-USD'],'yahoo') # Etherium

### med datum ###
# data = yf.download(['ETH-USD'], start='2010-01-01', end='2021-07-01', progress=False)
#################

data = yf.download(['ETH-USD'], progress=False)

ETH = pd.DataFrame(columns=['Adj Close','Volume'])
ETH['Adj Close'] = data['Adj Close'] #['ETH-USD']
ETH['Volume']=data['Volume'] #['ETH-USD']
print(ETH)


              Adj Close       Volume
Date                                
2015-08-07     2.772120       164329
2015-08-08     0.753325       674188
2015-08-09     0.701897       532170
2015-08-10     0.708448       405283
2015-08-11     1.067860      1463100
...                 ...          ...
2021-08-04  2724.619873  25038698173
2021-08-05  2827.328857  31057928075
2021-08-06  2890.941650  26528577879
2021-08-07  3157.238770  33081467129
2021-08-08  3145.158203  33735462912

[2190 rows x 2 columns]


In [ ]:

plt.figure(figsize=(14, 6))
plt.plot(ETH['Adj Close'].tail(60))
plt.xticks(rotation=45)
plt.show()

In [ ]:
### plot Adj Close ###
x=ETH.index

y1=ETH['Adj Close']
plt.figure(figsize=(14,6))
plt.title('ETH "Adjusted Close"')
plt.plot(x,y1, label='ETH')
plt.ylabel("$USD")
# plt.xlabel('Datum')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()
###
### plot Volume ###
x=ETH.index

y1=data['Volume'].copy()/1e07*0.5
plt.figure(figsize=(14,6))
plt.title('ETH "Volume"')
plt.plot(x,y1, label='ETH')
plt.ylabel("Antal")
# plt.xlabel('Datum')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
#.pct_change()
#ETH.drop('return_1',axis=1,inplace=True)

In [ ]:
# Nya kolumner
def proc_change(ny, gammal, dagar):
    ETH[ny+'_'+str(dagar)] = ETH[gammal].pct_change(dagar)
    ETH.dropna(inplace=True)
    

proc_change('ret','Adj Close',1)
proc_change('vol','Volume',1)

ETH['year']=pd.DatetimeIndex(ETH.index).year
ETH['month']=pd.DatetimeIndex(ETH.index).month
ETH['wday']=pd.DatetimeIndex(ETH.index).weekday
ETH['day']=pd.DatetimeIndex(ETH.index).day
ETH

In [ ]:
### plot return_1 ###
x=ETH.index

y1=ETH['ret_1'].copy()
plt.figure(figsize=(10,6))
plt.title('ETH "daily return"')
plt.plot(x,y1, label='ETH')
plt.ylabel("%")
# plt.xlabel('Datum')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

### plot vol_1 ###
x=ETH.index

y1=ETH['vol_1'].copy()
plt.figure(figsize=(10,6))
plt.title('ETH "daily volume change"')
plt.plot(x,y1, label='ETH')
plt.ylabel("%")
# plt.xlabel('Datum')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# plot genomsnittl return per månad
month_mean = ETH.groupby('month').ret_1.mean()
x=month_mean.index

y1=month_mean
y2=[0,0,0,0,0,0,0,0,0,0,0,0]
plt.figure(figsize=(10,6))
plt.title('ETH "monthly avg return"')
plt.scatter(x,y1,linewidths=5, label='avg return')
plt.plot(x,y2)
plt.ylabel("daily return")
plt.xlabel('Månad')
#plt.xticks(rotation=45)
#plt.legend(loc='upper left')
#plt.tight_layout()
plt.show()



In [ ]:
y1=ETH.groupby('wday').ret_1.mean()
x=y1.index
# y2=[0,0,0,0,0,0,0,0,0,0,0,0]
plt.figure(figsize=(10,6))
plt.title('ETH "avg return by weekday"')
plt.scatter(x,y1, linewidths=5, label='avg return')
# plt.plot(x,y2)
plt.ylabel("daily return")
plt.xlabel('Veckodag')
#plt.xticks(rotation=45)
#plt.legend(loc='upper left')
#plt.tight_layout()
plt.show()


In [ ]:
y1=ETH.groupby('day').ret_1.mean()
x=y1.index
# y2=[0,0,0,0,0,0,0,0,0,0,0,0]
plt.figure(figsize=(10,6))
plt.title('ETH "avg return by day of month"')
plt.scatter(x,y1, linewidths=5, label='avg return')
# plt.plot(x,y2)
plt.ylabel("daily return")
plt.xlabel('dag i måndaen')
#plt.xticks(rotation=45)
#plt.legend(loc='upper left')
#plt.tight_layout()
plt.show()



In [ ]:
# fler historiska kolumner
proc_change('ret','Adj Close',2)   
proc_change('ret','Adj Close',3)
proc_change('ret','Adj Close',4)
proc_change('ret','Adj Close',5)
proc_change('vol','Volume',2)
proc_change('vol','Volume',3)
proc_change('vol','Volume',4)
proc_change('vol','Volume',5)
ETH.dropna(inplace=True)    # Ta brt alla NaN som skapats av shift()
print(ETH.shape)

corr = ETH[2:].corr()
  
# Displaying dataframe as an heatmap 
# with diverging colourmap as coolwarm
corr.style.background_gradient('coolwarm')

In [ ]:
ETH.columns, ETH.index

In [ ]:
ETH.rename(columns={'vol_1':'vol_y', 'ret_1':'ret_y'},inplace=True)

mina_kolumner = ['Adj Close', 'ret_y', 'Volume', 'vol_y',
 'ret_2',
 'ret_3',
 'ret_4',
 'ret_5',
 'vol_2',
 'vol_3',
 'vol_4',
 'vol_5',
 'month',
 'year',
 'wday',
 'day']

# Kör en klassisk train/test för att bestämma iterations=x

In [ ]:
### gör split
def train_test_split(data,perc):
    print(max(data.index))
        # data.drop(data.index[len(data)-1],inplace=True)
    n = int(len(data) * (1-perc)) # split point
    train=data[:n].copy()
    train.dropna(inplace=True)
    test=data[n:].copy()
    test.dropna(inplace=True)
    return train,test # train and test

In [ ]:
# let the data include vol_y as input to ret_y-training
def run_train_test(train,test,cat_features):
    
    train_pool=Pool(train[mina_kolumner[3:]],train['ret_y'],cat_features=cat_features)  # include vol_y

    test_pool =Pool(test [mina_kolumner[3:]],test['ret_y'], cat_features=cat_features)
    
    model = CatBoostRegressor(iterations=3000,use_best_model=True, eval_metric='MAE',
             verbose=100, random_state=2021)

    model.fit(train_pool,use_best_model=True,eval_set=test_pool,early_stopping_rounds=200)
    pred = model.predict(test_pool)
    print('MAE',model.get_best_score()['validation']['MAE'])
    return model,pred

In [ ]:
train,test = train_test_split(ETH,0.33)
# använd 720 senaste dagarna (2 år) i train
train = train.tail(720)

model,pred = run_train_test(train,test,[])

## Vi väljer därför iterations=550

In [ ]:
############## Kolla hur Close-värdet och Volume ökar dramatiskt ###############

print(test[['Adj Close','Volume','ret_y','vol_y']])
ETH.ret_y.abs().describe()
#x

In [ ]:
test['pred_y'] = pred

In [ ]:
# Beräkna felet från 'ret_y'-skalan omräknat till 'Adj Close'-skalan
test['err'] = test['Adj Close'] - test.pred_y
test[['Adj Close','pred_y','err']].tail(10)

In [ ]:
print('mina_kolumner len:', len(mina_kolumner))
print('mina_kolumner:',mina_kolumner)
print('ETH.columns == mina_kolumner:',set(ETH.columns) == set(mina_kolumner))
ETH.columns

# Kör en walkthrough training

In [ ]:
# Denna är enbart för walkthrough - skapar test med endast en rad
def skapa_train_test(df,kolumner,fr_ix,antal_dagar,cat_features=[]):
    to_ix = fr_ix+antal_dagar
    train = df.iloc[fr_ix:to_ix]
    test  = df.iloc[to_ix:to_ix+1]
    
    cat_features = []
    return train, test, cat_features


In [ ]:
def walkthrough(df, kolumner, iterations=550):
    antal_träningsdagar=720  # Så många dagar ingår i träningsdata varje gång
    antal_preddagar=5        # Så många dagar framåt skall vi leverera predictions
    err=[]
    model = CatBoostRegressor(iterations=iterations,
            verbose=False, random_state=2021)
 
    blockdagar=antal_träningsdagar + antal_preddagar
    
    print(f'{blockdagar} dagar i varje train/predict omgång')
    fr_ix=int(len(ETH)/3)
    
    print(f'Starta från {str(fr_ix)} till {len(ETH)}. Träna i block om {antal_träningsdagar} dagar och predict {antal_preddagar} dagar varje gång')
    while fr_ix+antal_träningsdagar < len(ETH):
      train,test,cat_features = skapa_train_test(ETH,kolumner,fr_ix,antal_träningsdagar)
      train_pool=Pool(train[kolumner],train['ret_y'],cat_features=cat_features)
      test_pool =Pool(test [kolumner],test['ret_y'], cat_features=cat_features)
    
      model.fit(train_pool,use_best_model=False)
      pred = model.predict(test_pool)
      err.append(test.ret_y[0] - pred[0])
      
      Startdatum = ETH.iloc[fr_ix:fr_ix+1].index
      fr_ix += 1
      
      
    return model,err  

In [ ]:
### Körs här ###
kolumner = ETH.columns[3:]  # ta bort Adj Close, Volume, ret_y men inte vol_y
model,err = walkthrough(ETH,kolumner)

print('Finished')

In [ ]:
# translate the predicted ret_y back to Adj Close scale
def ret_y2Close(predicted_ret_y, previous_AdjClose):
    predicted_AdjClose = (1-predicted_ret_y) * previous_AdjClose
    return predicted_AdjClose

In [ ]:
# create models for both ret_y and vol_y
def create_models(data, kolumner, n_train=720, n_test=90, iterations=550):
    test=data.tail(n_test)
    train= data.iloc[-n_train-n_test:-n_test]
    print('shape train',train.shape)
    print('shape test',test.shape)    
    vmodel = CatBoostRegressor(iterations=iterations,
            verbose=False, random_state=2021)
    vmodel.fit(train[kolumner[2:]],y=train.vol_y,use_best_model=False) # train on 'kolumner' except vol_y and ret_y
    
    model = CatBoostRegressor(iterations=iterations,
            verbose=False, random_state=2021)
    model.fit(train[kolumner[1:]],y=train.ret_y,use_best_model=False) # includes vol_y for training
    
    vmodel.save_model('vmodel')
    model.save_model('model')
    
    return vmodel, model, test[kolumner], train[kolumner]

vmodel,model,test,train = create_models(ETH,ETH.columns[2:]) 

In [ ]:
def new_line(df_line, vol_y):
    from datetime import timedelta
    last_date = df_line.iloc[[-1]].index
    last_date = last_date + timedelta(days=1)
    last_row = df_line.copy().iloc[-1]
    
    data = {'vol_y': vol_y, 'vol_2':last_row.vol_y, 'vol_3':last_row.vol_2, 'vol_4':last_row.vol_3, 'vol_5':last_row.vol_4,
                            'ret_2':last_row.ret_y, 'ret_3':last_row.ret_2, 'ret_4':last_row.ret_3, 'ret_5':last_row.ret_4}
    df=df_line.copy().append(pd.DataFrame(data,index=[last_date[0]]))
    df['year']=pd.DatetimeIndex(df.index).year
    df['month']=pd.DatetimeIndex(df.index).month
    df['wday']=pd.DatetimeIndex(df.index).weekday
    df['day']=pd.DatetimeIndex(df.index).day
    return df

In [ ]:
def upd_line(df,ret_y):
    df.iat[-1,0]= ret_y
    
    return df

In [ ]:
# använd model till prediktion n dagar framåt
def predict_n_days(model, vmodel, df_line):
    
    kolumner = df_line.columns
    df=df_line.copy()
    for i in range(5):
      vol_y = vmodel.predict(df.iloc[-1][kolumner])
      df = new_line(df,vol_y)
      
      ret_y = model.predict(df.iloc[-1][kolumner])
      df = upd_line(df,ret_y)

    return df

In [ ]:
print('test kolumns:', test.columns)
predict_n_days(model,vmodel,test.copy().iloc[-1:][test.columns[2:]])

In [ ]:
from catboost.utils import eval_metric
train_pool,test_pool, cat_features = skapa_train_test(ETH,0,720)
model = CatBoostRegressor(iterations=300,
            verbose=False, random_state=2021)
 
model.fit(train_pool,use_best_model=False)
test  = ETH.iloc[720:730][['return_2','return_3','return_4']]

print(test)
pred = model.predict(test)

y=ETH.iloc[720:730][['ret_y']]
print('y',y.values)
print('pred',pred)
MAE = eval_metric(y.values,pred, 'MAE')
MAE


# Kör en total omgång

## prepare

### Läs in ETH och lägg till nya kolumner

In [5]:
# !Pip install ta

In [4]:
# !pip install ta
import pandas as pd

from datetime import datetime as dt
import yfinance as yf

# import pandas_datareader.data as web
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from catboost import CatBoostRegressor,Pool,utils

import time
plt.style.use('fivethirtyeight')
pd.set_option('display.width',200)


In [ ]:
data = yf.download(['ETH-USD'], progress=False)

ETH = pd.DataFrame(columns=['Adj Close','Volume'])
ETH['Adj Close'] = data['Adj Close'] #['ETH-USD']
ETH['Volume']=data['Volume'] #['ETH-USD']
print(ETH.tail())


In [ ]:
# Nya kolumner
def proc_change(df, ny, gammal, dagar):
    df[ny+'_'+str(dagar)] = df[gammal].pct_change(dagar)
    # df.drop(df.iloc[:dagar],inplace=True)
    
for i in range (10):
    proc_change(ETH,'ret','Adj Close',i+1)
    proc_change(ETH,'vol','Volume',i+1)
    
ETH.dropna(inplace=True)    # Ta brt alla NaN som skapats av shift()
print(ETH.shape)

ETH['year']=pd.DatetimeIndex(ETH.index).year
ETH['month']=pd.DatetimeIndex(ETH.index).month
ETH['wday']=pd.DatetimeIndex(ETH.index).weekday
ETH['day']=pd.DatetimeIndex(ETH.index).day

ETH.rename(columns={'vol_1':'vol_y', 'ret_1':'ret_y'},inplace=True)

ETH.tail()

### train/test split

In [ ]:
def train_test_split(data,perc):
    print('sista datum:',max(data.index))
        # data.drop(data.index[len(data)-1],inplace=True)
    n = int(len(data) * (1-perc)) # split point
    train=data[:n].copy()
    train.dropna(inplace=True)
    test=data[n:].copy()
    test.dropna(inplace=True)
    return train,test # train and test

### fit

In [ ]:
def set_kolumner():
    rkolumner = ['vol_y', 'ret_2', 'vol_2', 'ret_3', 'vol_3', 'ret_4', 'vol_4', 'ret_5', 'vol_5', 
                'ret_6', 'vol_6', 'ret_7', 'vol_7', 'ret_8', 'vol_8', 'ret_9', 'vol_9', 'ret_10', 
                'vol_10', 'year', 'month', 'wday', 'day']
    vkolumner = rkolumner[1:]
    
    return vkolumner, rkolumner

In [ ]:
# let the data include vol_y as input to ret_y-training
def fit_data(train, iterations=550):

    vkolumner, rkolumner = set_kolumner()
    
    ret_y=train.ret_y
    vol_y=train.vol_y

    print('shape train av ret_y',train[rkolumner].shape)   
    print('shape train av vol_y',train[vkolumner].shape)   
    
    vmodel = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    vmodel.fit(train[vkolumner],y=vol_y,use_best_model=False) # train on 'kolumner' except vol_y and ret_y
    
    model = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    model.fit(train[rkolumner],y=ret_y,use_best_model=False) # includes vol_y for training
    
    vmodel.save_model('vmodel')
    model.save_model('model')
    
    return vmodel, model

In [ ]:
train,test = train_test_split(ETH,0.33)

# spara 30 dagar för validering
train = train.tail(720)  # använd de 720 sista dagarna (2 år)
vmodel, model = fit_data(train)

:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/data_providers.cpp:533: Currently only multi-regression and survival objectives work with multidimensional target

## Kör en/flera predict

### skapa nya kolumner för pred_ret_y, pred_vol_y, pred_Volume, Pred_AdjCLoe fill med NA

In [ ]:
for_pred= test.copy()
for_pred['ret_y'] = None
for_pred['vol_y'] = None
for_pred['pred_Volume'] = None
for_pred['pred_AdjClose'] = None

In [ ]:
# translate the predicted ret_y and vol_y back to Adj Close and Volume scale
def transl_ret_y(predicted_ret_y, predicted_vol_y, previous_AdjClose, previous_Volume):
    predicted_AdjClose = (1+predicted_ret_y) * previous_AdjClose
    predicted_Volume = (1+predicted_vol_y) * previous_Volume
    return predicted_AdjClose, predicted_Volume

In [ ]:
# kör n predictions i framåt i tiden
def pred_grupp(df, model, vmodel, n, startix):
    
    print(f'pred_grupp:  df.shape={df.shape} startix={startix}')
    vkolumner, rkolumner = set_kolumner()
    for i in range(n):
        l=df.iloc[startix+i:startix+i+1].index
        vol_y = vmodel.predict(df[vkolumner].iloc[startix+i])
        # print('vol_y',vol_y)
        df.loc[l,'vol_y'] = vol_y
        # print(df.loc[l,'vol_y'])
        ret_y = model.predict(df[rkolumner].iloc[startix+i])
        
        df.loc[l,'ret_y'] = ret_y
        
        # översätt till rätt skala
        predicted_AdjClose, predicted_Volume = transl_ret_y(ret_y, vol_y, df.iloc[startix+i-1]['Adj Close'],df.iloc[startix+i-1].Volume)
        
        df.loc[l,'pred_AdjClose'] = predicted_AdjClose
        df.loc[l,'pred_Volume'] = predicted_Volume
        
    return df

### Plocka ut några datum för enkelt test

In [ ]:
print(len(for_pred))
l=len(for_pred)
testix=[605,625,645,665,l-5]

In [ ]:
n=5
for ix in testix:
    for_pred = pred_grupp(for_pred,model,vmodel,n,ix)

for_pred[['Adj Close','Volume','pred_AdjClose','pred_Volume']].tail(60)

### plot result

In [ ]:
### plot Adj Close ###

x=for_pred.tail(80).index

y1=for_pred['Adj Close'].tail(80)
y2=for_pred['pred_AdjClose'].tail(80)
plt.figure(figsize=(14,6))
plt.title('ETH "Adjusted Close"')
plt.plot(x,y1, label='ETH')
plt.ylabel("$USD")
plt.scatter(x,y2, linewidths=2, color='r',label='predicted')
# plt.xlabel('Datum')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()
